In [151]:
import numpy as np
import pandas as pd
from pandas import DataFrame,Series

from keras import models
from keras.models import Sequential
from keras.models import Model
from keras.models import load_model
from keras import backend as K

from keras import layers
from keras.layers import Layer
from keras.layers import Input,Dense,Flatten,Embedding,Permute,Dot,Reshape
from keras.layers.convolutional import Conv1D,MaxPooling1D,MaxPooling2D
from keras.layers import Dropout
from keras.layers import LSTM,GRU

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from keras.preprocessing import sequence
from keras.utils import np_utils

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import cv2

import os

In [85]:
file_path = ".\\train"
train_list = os.listdir(file_path)

In [86]:
img = cv2.imread(file_path +'\\'+ train_list[1],0)

In [9]:
total_set = dict()

for i in train_list[1:]:
    temp = cv2.imread(file_path + "\\" + i,0)
    total_set[i] = temp.copy()

AttributeError: 'NoneType' object has no attribute 'copy'

In [110]:
np.shape(total_set[list(total_set.keys())[1]])

(256, 1600)

## 단순 컨볼루션

In [164]:
inputs = layers.Input(shape=(1,256,1600),name = "Feed_Sentence")

conv_1 = layers.Conv2D(256,(3,3),padding="valid",activation = "relu",data_format = "channels_first")(inputs)
conv_1 = layers.Conv2D(256,(3,3),padding="valid",activation = "relu",data_format = "channels_first")(conv_1)

MaxPooling_layer = layers.MaxPooling2D((2,2))(conv_1)
Flatten_layer = layers.Flatten()(MaxPooling_layer)

outputs = layers.Dense(8,activation = "relu")(Flatten_layer)

models = Model(inputs = inputs,outputs = outputs)

In [165]:
models.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Feed_Sentence (InputLayer)   (None, 1, 256, 1600)      0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 256, 254, 1598)    2560      
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 256, 252, 1596)    590080    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 128, 126, 1596)    0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25740288)          0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 205922312 
Total params: 206,514,952
Trainable params: 206,514,952
Non-trainable params: 0
______________________________________________________________

In [ ]:
SVG(model_to_dot(conv_1,show_shapes = True)).create(prog="dot",format="svg")